### Importing required libraries

In [63]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import zipfile
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)
from IPython.display import display
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
import datetime as dt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier 
from sklearn.preprocessing import StandardScaler
from sklearn.mixture import GaussianMixture

Extracting Training and Test datasets from Zip files

In [6]:
with zipfile.ZipFile('data/Train.zip', 'r') as zip_ref:
    zip_ref.extractall('data/')

In [7]:
with zipfile.ZipFile('data/Test.zip', 'r') as zip_ref:
    zip_ref.extractall('data/Test/')

In [2]:
train = pd.read_excel('data/Train/train_Data.xlsx',    engine='openpyxl')
test = pd.read_excel('data/Test/test_Data.xlsx',    engine='openpyxl')

In [3]:
train.head()

,ID,Frequency,InstlmentMode,LoanStatus,PaymentMode,BranchID,Area,Tenure,AssetCost,AmountFinance,DisbursalAmount,EMI,DisbursalDate,MaturityDAte,AuthDate,AssetID,ManufacturerID,SupplierID,LTV,SEX,AGE,MonthlyIncome,City,State,ZiPCODE,Top-up Month
0,1,Monthly,Arrear,Closed,PDC_E,1,NaN,48,450000,275000.00,275000.00,24000.00,2012-02-10,2016-01-15,2012-02-10,4022465,1568,21946,61.11,M,49.00,35833.33,RAISEN,MADHYA PRADESH,464993.00,> 48 Months
1,2,Monthly,Advance,Closed,PDC,333,BHOPAL,47,485000,350000.00,350000.00,10500.00,2012-03-31,2016-02-15,2012-03-31,4681175,1062,34802,70.00,M,23.00,666.67,SEHORE,MADHYA PRADESH,466001.00,No Top-up Service
2,3,Quatrly,Arrear,Active,Direct Debit,1,NaN,68,690000,519728.00,519728.00,38300.00,2017-06-17,2023-02-10,2017-06-17,25328146,1060,127335,69.77,M,39.00,45257.00,BHOPAL,MADHYA PRADESH,462030.00,12-18 Months
3,7,Monthly,Advance,Closed,Billed,125,GUNA,48,480000,400000.00,400000.00,11600.00,2013-11-29,2017-11-10,2013-11-29,13021591,1060,25094,80.92,M,24.00,20833.33,ASHOK NAGAR,MADHYA PRADESH,473335.00,> 48 Months
4,8,Monthly,Arrear,Closed,Billed,152,BILASPUR,44,619265,440000.00,440000.00,15000.00,2011-12-08,2015-07-05,2011-12-08,3291320,1046,21853,71.05,M,56.00,27313.67,BILASPUR,CHATTISGARH,495442.00,36-48 Months


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128655 entries, 0 to 128654
Data columns (total 26 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   ID               128655 non-null  int64         
 1   Frequency        128655 non-null  object        
 2   InstlmentMode    128655 non-null  object        
 3   LoanStatus       128655 non-null  object        
 4   PaymentMode      128655 non-null  object        
 5   BranchID         128655 non-null  int64         
 6   Area             117002 non-null  object        
 7   Tenure           128655 non-null  int64         
 8   AssetCost        128655 non-null  int64         
 9   AmountFinance    128655 non-null  float64       
 10  DisbursalAmount  128655 non-null  float64       
 11  EMI              128655 non-null  float64       
 12  DisbursalDate    128655 non-null  datetime64[ns]
 13  MaturityDAte     128654 non-null  datetime64[ns]
 14  AuthDate         128

In [5]:
train.describe()

,ID,BranchID,Tenure,AssetCost,AmountFinance,DisbursalAmount,EMI,AssetID,ManufacturerID,SupplierID,LTV,AGE,MonthlyIncome,ZiPCODE
count,128655.00,128655.00,128655.00,128655.00,128655.00,128655.00,128655.00,128655.00,128655.00,128655.00,128655.00,128596.00,128421.00,128283.00
mean,70965.33,166.29,42.46,555024.68,348309.63,347930.57,55072.76,16397728.99,1171.33,56551.23,59.13,40.66,50323.60,427931.10
std,41762.78,115.84,23.53,108303.55,105545.34,105319.84,28910.11,9539540.25,257.79,36399.36,15.54,11.71,1724607.36,175704.36
min,1.00,1.00,5.00,200000.00,50000.00,2894.00,0.00,422271.00,1019.00,5879.00,-1.38,18.00,0.00,110000.00
25%,34408.50,50.00,36.00,500000.00,290000.00,290000.00,32500.00,9438948.50,1046.00,24093.00,49.14,31.00,25000.00,304804.00
50%,70988.00,152.00,36.00,550735.00,350000.00,350000.00,59700.00,15133927.00,1062.00,39146.00,60.78,40.00,35833.33,458553.00
75%,106549.50,274.00,48.00,611000.00,410000.00,410000.00,73800.00,25333957.50,1186.00,87034.00,70.59,49.00,50000.00,521131.00
max,143395.00,424.00,501.00,2250000.00,1308351.00,1308351.00,460000.00,37066666.00,3473.00,145518.00,100.00,90.00,617477500.00,855456.00


In [6]:
test.head()

,ID,Frequency,InstlmentMode,LoanStatus,PaymentMode,BranchID,Area,Tenure,AssetCost,AmountFinance,DisbursalAmount,EMI,DisbursalDate,MaturityDAte,AuthDate,AssetID,ManufacturerID,SupplierID,LTV,SEX,AGE,MonthlyIncome,City,State,ZiPCODE
0,4,Monthly,Advance,Closed,PDC_E,2,GUNA,46,480000,365000.00,365000.00,1000.00,2011-12-29,2015-10-05,2011-12-29,3524747,1046.00,22354,75.83,M,50.00,32069.00,GUNA,MADHYA PRADESH,473001.00
1,5,Monthly,Advance,Closed,PDC,2,GUNA,45,480000,285000.00,285000.00,9300.00,2012-04-28,2016-01-01,2012-04-28,4985862,1046.00,22354,57.44,M,35.00,25000.00,GUNA,MADHYA PRADESH,473001.00
2,6,Quatrly,Arrear,Closed,PDC,2,GUNA,48,580000,400000.00,400000.00,35800.00,2013-10-22,2017-09-10,2013-10-22,12881783,1060.00,65929,68.97,M,37.00,23333.33,GUNA,MADHYA PRADESH,473001.00
3,25,Half Yearly,Arrear,Closed,Billed,154,NaN,36,725000,500000.00,500000.00,52000.00,2013-05-21,2016-02-10,2013-05-21,10613776,1049.00,68597,61.79,M,55.00,91666.67,MAHASAMUND,CHATTISGARH,493558.00
4,119,Quatrly,Arrear,Closed,PDC,194,CUTTACK,48,617000,400000.00,400000.00,35400.00,2012-11-30,2016-12-05,2012-11-30,7911110,1049.00,38493,59.09,M,48.00,12500.00,JAGATSINGHAPUR,ORISSA,754137.00


In [7]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14745 entries, 0 to 14744
Data columns (total 25 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   ID               14745 non-null  int64         
 1   Frequency        14745 non-null  object        
 2   InstlmentMode    14745 non-null  object        
 3   LoanStatus       14745 non-null  object        
 4   PaymentMode      14745 non-null  object        
 5   BranchID         14745 non-null  int64         
 6   Area             14199 non-null  object        
 7   Tenure           14745 non-null  int64         
 8   AssetCost        14745 non-null  int64         
 9   AmountFinance    14745 non-null  float64       
 10  DisbursalAmount  14745 non-null  float64       
 11  EMI              14745 non-null  float64       
 12  DisbursalDate    14745 non-null  datetime64[ns]
 13  MaturityDAte     14745 non-null  datetime64[ns]
 14  AuthDate         14745 non-null  datet

In [8]:
test.describe()

,ID,BranchID,Tenure,AssetCost,AmountFinance,DisbursalAmount,EMI,AssetID,ManufacturerID,SupplierID,LTV,AGE,MonthlyIncome,ZiPCODE
count,14745.00,14745.00,14745.00,14745.00,14745.00,14745.00,14745.00,14745.00,14743.00,14745.00,14745.00,14738.00,14731.00,14744.00
mean,78115.13,199.35,44.95,580089.54,375929.72,375628.06,58910.21,16700878.29,1129.11,54897.40,61.26,40.23,45084.91,398320.51
std,37441.42,110.98,20.21,92410.81,101345.03,101233.04,27551.69,7279988.23,237.61,32752.10,16.07,11.60,87199.04,143284.70
min,4.00,1.00,10.00,215000.00,80000.00,8433.00,100.00,1827219.00,1019.00,20055.00,6.93,18.00,0.00,110043.00
25%,54107.00,85.00,36.00,525000.00,301002.00,300945.00,37700.00,12823268.00,1046.00,26482.00,50.96,31.00,25000.00,323024.00
50%,78414.00,199.00,46.00,572000.00,394405.00,394000.00,63700.00,14637999.00,1049.00,44255.00,62.81,39.00,33333.33,431122.00
75%,113859.00,302.00,48.00,635000.00,450000.00,450000.00,75701.00,23562146.00,1186.00,80465.00,73.33,49.00,50000.00,473101.00
max,143400.00,424.00,282.00,1630000.00,1130000.00,1130000.00,360960.00,34024952.00,3473.00,142782.00,98.20,85.00,7975000.00,855101.00


### Preprocessing data for Training

In [9]:
train.columns

Index(['ID', 'Frequency', 'InstlmentMode', 'LoanStatus', 'PaymentMode',
       'BranchID', 'Area', 'Tenure', 'AssetCost', 'AmountFinance',
       'DisbursalAmount', 'EMI', 'DisbursalDate', 'MaturityDAte', 'AuthDate',
       'AssetID', 'ManufacturerID', 'SupplierID', 'LTV', 'SEX', 'AGE',
       'MonthlyIncome', 'City', 'State', 'ZiPCODE', 'Top-up Month'],
      dtype='object')

In [10]:
train1= train.drop(['Area','City','State'], axis=1)

In [11]:
train1.head()

,ID,Frequency,InstlmentMode,LoanStatus,PaymentMode,BranchID,Tenure,AssetCost,AmountFinance,DisbursalAmount,EMI,DisbursalDate,MaturityDAte,AuthDate,AssetID,ManufacturerID,SupplierID,LTV,SEX,AGE,MonthlyIncome,ZiPCODE,Top-up Month
0,1,Monthly,Arrear,Closed,PDC_E,1,48,450000,275000.00,275000.00,24000.00,2012-02-10,2016-01-15,2012-02-10,4022465,1568,21946,61.11,M,49.00,35833.33,464993.00,> 48 Months
1,2,Monthly,Advance,Closed,PDC,333,47,485000,350000.00,350000.00,10500.00,2012-03-31,2016-02-15,2012-03-31,4681175,1062,34802,70.00,M,23.00,666.67,466001.00,No Top-up Service
2,3,Quatrly,Arrear,Active,Direct Debit,1,68,690000,519728.00,519728.00,38300.00,2017-06-17,2023-02-10,2017-06-17,25328146,1060,127335,69.77,M,39.00,45257.00,462030.00,12-18 Months
3,7,Monthly,Advance,Closed,Billed,125,48,480000,400000.00,400000.00,11600.00,2013-11-29,2017-11-10,2013-11-29,13021591,1060,25094,80.92,M,24.00,20833.33,473335.00,> 48 Months
4,8,Monthly,Arrear,Closed,Billed,152,44,619265,440000.00,440000.00,15000.00,2011-12-08,2015-07-05,2011-12-08,3291320,1046,21853,71.05,M,56.00,27313.67,495442.00,36-48 Months


In [12]:
train2 = train1.dropna()
train2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 128043 entries, 0 to 128654
Data columns (total 23 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   ID               128043 non-null  int64         
 1   Frequency        128043 non-null  object        
 2   InstlmentMode    128043 non-null  object        
 3   LoanStatus       128043 non-null  object        
 4   PaymentMode      128043 non-null  object        
 5   BranchID         128043 non-null  int64         
 6   Tenure           128043 non-null  int64         
 7   AssetCost        128043 non-null  int64         
 8   AmountFinance    128043 non-null  float64       
 9   DisbursalAmount  128043 non-null  float64       
 10  EMI              128043 non-null  float64       
 11  DisbursalDate    128043 non-null  datetime64[ns]
 12  MaturityDAte     128043 non-null  datetime64[ns]
 13  AuthDate         128043 non-null  datetime64[ns]
 14  AssetID          128

In [13]:
train3  = train2.rename(columns={'Top-up Month': 'Top-up-Month'})

In [15]:
colnames=['Frequency','InstlmentMode','LoanStatus','PaymentMode','SEX']
colnames

['Frequency', 'InstlmentMode', 'LoanStatus', 'PaymentMode', 'SEX']

In [16]:
train3[colnames]

,Frequency,InstlmentMode,LoanStatus,PaymentMode,SEX
0,Monthly,Arrear,Closed,PDC_E,M
1,Monthly,Advance,Closed,PDC,M
2,Quatrly,Arrear,Active,Direct Debit,M
3,Monthly,Advance,Closed,Billed,M
4,Monthly,Arrear,Closed,Billed,M
...,...,...,...,...,...
128650,Half Yearly,Arrear,Closed,Direct Debit,M
128651,Half Yearly,Arrear,Closed,Direct Debit,M
128652,Monthly,Arrear,Active,Direct Debit,M
128653,Half Yearly,Arrear,Active,Direct Debit,M


In [17]:
le = LabelEncoder()

In [18]:
train3[['Frequency1','InstlmentMode1','LoanStatus1','PaymentMode1','SEX1']]=train3[colnames].apply(le.fit_transform)

In [19]:
train3.head()

,ID,Frequency,InstlmentMode,LoanStatus,PaymentMode,BranchID,Tenure,AssetCost,AmountFinance,DisbursalAmount,EMI,DisbursalDate,MaturityDAte,AuthDate,AssetID,ManufacturerID,SupplierID,LTV,SEX,AGE,MonthlyIncome,ZiPCODE,Top-up-Month,Frequency1,InstlmentMode1,LoanStatus1,PaymentMode1,SEX1
0,1,Monthly,Arrear,Closed,PDC_E,1,48,450000,275000.00,275000.00,24000.00,2012-02-10,2016-01-15,2012-02-10,4022465,1568,21946,61.11,M,49.00,35833.33,464993.00,> 48 Months,2,1,1,9,1
1,2,Monthly,Advance,Closed,PDC,333,47,485000,350000.00,350000.00,10500.00,2012-03-31,2016-02-15,2012-03-31,4681175,1062,34802,70.00,M,23.00,666.67,466001.00,No Top-up Service,2,0,1,7,1
2,3,Quatrly,Arrear,Active,Direct Debit,1,68,690000,519728.00,519728.00,38300.00,2017-06-17,2023-02-10,2017-06-17,25328146,1060,127335,69.77,M,39.00,45257.00,462030.00,12-18 Months,3,1,0,3,1
3,7,Monthly,Advance,Closed,Billed,125,48,480000,400000.00,400000.00,11600.00,2013-11-29,2017-11-10,2013-11-29,13021591,1060,25094,80.92,M,24.00,20833.33,473335.00,> 48 Months,2,0,1,1,1
4,8,Monthly,Arrear,Closed,Billed,152,44,619265,440000.00,440000.00,15000.00,2011-12-08,2015-07-05,2011-12-08,3291320,1046,21853,71.05,M,56.00,27313.67,495442.00,36-48 Months,2,1,1,1,1


In [20]:
train4=train3.drop(colnames, axis=1)
train4.head()

,ID,BranchID,Tenure,AssetCost,AmountFinance,DisbursalAmount,EMI,DisbursalDate,MaturityDAte,AuthDate,AssetID,ManufacturerID,SupplierID,LTV,AGE,MonthlyIncome,ZiPCODE,Top-up-Month,Frequency1,InstlmentMode1,LoanStatus1,PaymentMode1,SEX1
0,1,1,48,450000,275000.00,275000.00,24000.00,2012-02-10,2016-01-15,2012-02-10,4022465,1568,21946,61.11,49.00,35833.33,464993.00,> 48 Months,2,1,1,9,1
1,2,333,47,485000,350000.00,350000.00,10500.00,2012-03-31,2016-02-15,2012-03-31,4681175,1062,34802,70.00,23.00,666.67,466001.00,No Top-up Service,2,0,1,7,1
2,3,1,68,690000,519728.00,519728.00,38300.00,2017-06-17,2023-02-10,2017-06-17,25328146,1060,127335,69.77,39.00,45257.00,462030.00,12-18 Months,3,1,0,3,1
3,7,125,48,480000,400000.00,400000.00,11600.00,2013-11-29,2017-11-10,2013-11-29,13021591,1060,25094,80.92,24.00,20833.33,473335.00,> 48 Months,2,0,1,1,1
4,8,152,44,619265,440000.00,440000.00,15000.00,2011-12-08,2015-07-05,2011-12-08,3291320,1046,21853,71.05,56.00,27313.67,495442.00,36-48 Months,2,1,1,1,1


In [21]:
train4.info()
train4.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 128043 entries, 0 to 128654
Data columns (total 23 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   ID               128043 non-null  int64         
 1   BranchID         128043 non-null  int64         
 2   Tenure           128043 non-null  int64         
 3   AssetCost        128043 non-null  int64         
 4   AmountFinance    128043 non-null  float64       
 5   DisbursalAmount  128043 non-null  float64       
 6   EMI              128043 non-null  float64       
 7   DisbursalDate    128043 non-null  datetime64[ns]
 8   MaturityDAte     128043 non-null  datetime64[ns]
 9   AuthDate         128043 non-null  datetime64[ns]
 10  AssetID          128043 non-null  int64         
 11  ManufacturerID   128043 non-null  int64         
 12  SupplierID       128043 non-null  int64         
 13  LTV              128043 non-null  float64       
 14  AGE              128

,ID,BranchID,Tenure,AssetCost,AmountFinance,DisbursalAmount,EMI,DisbursalDate,MaturityDAte,AuthDate,AssetID,ManufacturerID,SupplierID,LTV,AGE,MonthlyIncome,ZiPCODE,Top-up-Month,Frequency1,InstlmentMode1,LoanStatus1,PaymentMode1,SEX1
0,1,1,48,450000,275000.00,275000.00,24000.00,2012-02-10,2016-01-15,2012-02-10,4022465,1568,21946,61.11,49.00,35833.33,464993.00,> 48 Months,2,1,1,9,1
1,2,333,47,485000,350000.00,350000.00,10500.00,2012-03-31,2016-02-15,2012-03-31,4681175,1062,34802,70.00,23.00,666.67,466001.00,No Top-up Service,2,0,1,7,1
2,3,1,68,690000,519728.00,519728.00,38300.00,2017-06-17,2023-02-10,2017-06-17,25328146,1060,127335,69.77,39.00,45257.00,462030.00,12-18 Months,3,1,0,3,1
3,7,125,48,480000,400000.00,400000.00,11600.00,2013-11-29,2017-11-10,2013-11-29,13021591,1060,25094,80.92,24.00,20833.33,473335.00,> 48 Months,2,0,1,1,1
4,8,152,44,619265,440000.00,440000.00,15000.00,2011-12-08,2015-07-05,2011-12-08,3291320,1046,21853,71.05,56.00,27313.67,495442.00,36-48 Months,2,1,1,1,1


In [22]:
train4['DisbursalDate1']=train4['DisbursalDate'].map(dt.datetime.toordinal)

In [23]:
train4['MaturityDAte1']=train4['MaturityDAte'].map(dt.datetime.toordinal)
train4['AuthDate1']=train4['AuthDate'].map(dt.datetime.toordinal)
train4.head()

,ID,BranchID,Tenure,AssetCost,AmountFinance,DisbursalAmount,EMI,DisbursalDate,MaturityDAte,AuthDate,AssetID,ManufacturerID,SupplierID,LTV,AGE,MonthlyIncome,ZiPCODE,Top-up-Month,Frequency1,InstlmentMode1,LoanStatus1,PaymentMode1,SEX1,DisbursalDate1,MaturityDAte1,AuthDate1
0,1,1,48,450000,275000.00,275000.00,24000.00,2012-02-10,2016-01-15,2012-02-10,4022465,1568,21946,61.11,49.00,35833.33,464993.00,> 48 Months,2,1,1,9,1,734543,735978,734543
1,2,333,47,485000,350000.00,350000.00,10500.00,2012-03-31,2016-02-15,2012-03-31,4681175,1062,34802,70.00,23.00,666.67,466001.00,No Top-up Service,2,0,1,7,1,734593,736009,734593
2,3,1,68,690000,519728.00,519728.00,38300.00,2017-06-17,2023-02-10,2017-06-17,25328146,1060,127335,69.77,39.00,45257.00,462030.00,12-18 Months,3,1,0,3,1,736497,738561,736497
3,7,125,48,480000,400000.00,400000.00,11600.00,2013-11-29,2017-11-10,2013-11-29,13021591,1060,25094,80.92,24.00,20833.33,473335.00,> 48 Months,2,0,1,1,1,735201,736643,735201
4,8,152,44,619265,440000.00,440000.00,15000.00,2011-12-08,2015-07-05,2011-12-08,3291320,1046,21853,71.05,56.00,27313.67,495442.00,36-48 Months,2,1,1,1,1,734479,735784,734479


In [24]:
train5 = train4.drop(['MaturityDAte','AuthDate','DisbursalDate'],axis=1)
train5.head()

,ID,BranchID,Tenure,AssetCost,AmountFinance,DisbursalAmount,EMI,AssetID,ManufacturerID,SupplierID,LTV,AGE,MonthlyIncome,ZiPCODE,Top-up-Month,Frequency1,InstlmentMode1,LoanStatus1,PaymentMode1,SEX1,DisbursalDate1,MaturityDAte1,AuthDate1
0,1,1,48,450000,275000.00,275000.00,24000.00,4022465,1568,21946,61.11,49.00,35833.33,464993.00,> 48 Months,2,1,1,9,1,734543,735978,734543
1,2,333,47,485000,350000.00,350000.00,10500.00,4681175,1062,34802,70.00,23.00,666.67,466001.00,No Top-up Service,2,0,1,7,1,734593,736009,734593
2,3,1,68,690000,519728.00,519728.00,38300.00,25328146,1060,127335,69.77,39.00,45257.00,462030.00,12-18 Months,3,1,0,3,1,736497,738561,736497
3,7,125,48,480000,400000.00,400000.00,11600.00,13021591,1060,25094,80.92,24.00,20833.33,473335.00,> 48 Months,2,0,1,1,1,735201,736643,735201
4,8,152,44,619265,440000.00,440000.00,15000.00,3291320,1046,21853,71.05,56.00,27313.67,495442.00,36-48 Months,2,1,1,1,1,734479,735784,734479


In [25]:
train5.shape

(128043, 23)

### Training the model

In [26]:
X = train5.loc[:, train5.columns != 'Top-up-Month']
X

,ID,BranchID,Tenure,AssetCost,AmountFinance,DisbursalAmount,EMI,AssetID,ManufacturerID,SupplierID,LTV,AGE,MonthlyIncome,ZiPCODE,Frequency1,InstlmentMode1,LoanStatus1,PaymentMode1,SEX1,DisbursalDate1,MaturityDAte1,AuthDate1
0,1,1,48,450000,275000.00,275000.00,24000.00,4022465,1568,21946,61.11,49.00,35833.33,464993.00,2,1,1,9,1,734543,735978,734543
1,2,333,47,485000,350000.00,350000.00,10500.00,4681175,1062,34802,70.00,23.00,666.67,466001.00,2,0,1,7,1,734593,736009,734593
2,3,1,68,690000,519728.00,519728.00,38300.00,25328146,1060,127335,69.77,39.00,45257.00,462030.00,3,1,0,3,1,736497,738561,736497
3,7,125,48,480000,400000.00,400000.00,11600.00,13021591,1060,25094,80.92,24.00,20833.33,473335.00,2,0,1,1,1,735201,736643,735201
4,8,152,44,619265,440000.00,440000.00,15000.00,3291320,1046,21853,71.05,56.00,27313.67,495442.00,2,1,1,1,1,734479,735784,734479
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128650,143390,424,24,470000,265601.00,265601.00,76800.00,31286914,1568,48879,40.17,25.00,65333.33,131403.00,1,1,1,3,1,736958,737581,736959
128651,143391,424,24,460000,275630.00,275630.00,80100.00,31295422,1568,48879,59.92,25.00,83333.33,131403.00,1,1,1,3,1,736959,737581,736959
128652,143393,424,23,545000,300733.00,300733.00,15277.00,32145629,1568,44118,52.38,36.00,248500.00,131024.00,2,1,0,3,1,737021,737734,737021
128653,143394,424,35,350000,250962.00,250962.00,74341.00,32509866,1568,48879,50.37,37.00,84500.00,131103.00,1,1,0,3,1,737048,737946,737048


 ### Standard Scaler

In [28]:
scaling=StandardScaler()
X=scaling.fit_transform(X)
X

array([[-1.70472554, -1.43032727,  0.23329656, ..., -1.22696703,
        -0.88338099, -1.22722625],
       [-1.70470157,  1.43601352,  0.19084715, ..., -1.1715629 ,
        -0.85194903, -1.17181954],
       [-1.70467761, -1.43032727,  1.08228481, ...,  0.93822646,
         1.73561155,  0.93806783],
       ...,
       [ 1.73147797,  2.22166718, -0.82793876, ...,  1.51886177,
         0.89708781,  1.51873011],
       [ 1.73150193,  2.22166718, -0.31854581, ...,  1.54878   ,
         1.1120419 ,  1.54864973],
       [ 1.73152589,  2.22166718, -0.78548934, ...,  1.56096891,
         0.89708781,  1.5608392 ]])

In [29]:
Y = train5.iloc[:,-9]

In [30]:
Y.unique()

array([' > 48 Months', 'No Top-up Service', '12-18 Months',
       '36-48 Months', '18-24 Months', '24-30 Months', '30-36 Months'],
      dtype=object)

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [32]:
classifier = xgb.XGBClassifier()

In [33]:
classifier.fit(X_train, y_train)

c:\users\vagundu\appdata\local\r-mini~1\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:50:18] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [34]:
pd.DataFrame(classifier.feature_importances_.reshape(1, -1), columns=X.columns)

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [35]:
y_pred = classifier.predict(X_test)

In [36]:
np.unique(y_pred)

array([' > 48 Months', '12-18 Months', '18-24 Months', '24-30 Months',
       '30-36 Months', '36-48 Months', 'No Top-up Service'], dtype=object)

In [38]:
unique_elements, counts_elements = np.unique(y_pred, return_counts=True)
print(np.asarray((unique_elements, counts_elements)))

[[' > 48 Months' '12-18 Months' '18-24 Months' '24-30 Months'
  '30-36 Months' '36-48 Months' 'No Top-up Service']
 [36 5 23 36 7 12 31892]]


In [39]:
accuracy_score(y_test, y_pred)


0.8287463684358501

### Gaussian Algorithm

In [72]:
np.unique(y_train)

array([' > 48 Months', '12-18 Months', '18-24 Months', '24-30 Months',
       '30-36 Months', '36-48 Months', 'No Top-up Service'], dtype=object)

In [73]:
y_le = le.fit_transform(y_train)
np.unique(y_le)

array([0, 1, 2, 3, 4, 5, 6])

In [85]:
y_test_le = le.fit_transform(y_test)
np.unique(y_test_le)

array([0, 1, 2, 3, 4, 5, 6])

In [75]:
gm = GaussianMixture(n_components = 6)
gm.fit(X_train, y_le)
gm_predictions = gm.predict(X_test) 

In [76]:
np.unique(gm_predictions)

array([0, 1, 2, 3, 5], dtype=int64)

In [80]:
X_train.shape

(96032, 22)

In [81]:
y_train.shape

(96032,)

In [83]:
X_test.shape

(32011, 22)

In [84]:
y_test.shape

(32011,)

In [78]:
len(gm_predictions)

32011

In [86]:
accuracy_score(y_test_le,gm_predictions)

0.024554059542032425

### Preprocessing the test data to predict target

In [40]:
test.columns
test1= test.drop(['Area','City','State'], axis=1)

In [41]:
test2 = test1.fillna(test1.mean())
test2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14745 entries, 0 to 14744
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   ID               14745 non-null  int64         
 1   Frequency        14745 non-null  object        
 2   InstlmentMode    14745 non-null  object        
 3   LoanStatus       14745 non-null  object        
 4   PaymentMode      14745 non-null  object        
 5   BranchID         14745 non-null  int64         
 6   Tenure           14745 non-null  int64         
 7   AssetCost        14745 non-null  int64         
 8   AmountFinance    14745 non-null  float64       
 9   DisbursalAmount  14745 non-null  float64       
 10  EMI              14745 non-null  float64       
 11  DisbursalDate    14745 non-null  datetime64[ns]
 12  MaturityDAte     14745 non-null  datetime64[ns]
 13  AuthDate         14745 non-null  datetime64[ns]
 14  AssetID          14745 non-null  int64

<ipython-input-41-bb77e1df4259>:1: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  test2 = test1.fillna(test1.mean())


In [42]:
test2.SEX = test2.SEX.fillna('M')
test2[colnames]

,Frequency,InstlmentMode,LoanStatus,PaymentMode,SEX
0,Monthly,Advance,Closed,PDC_E,M
1,Monthly,Advance,Closed,PDC,M
2,Quatrly,Arrear,Closed,PDC,M
3,Half Yearly,Arrear,Closed,Billed,M
4,Quatrly,Arrear,Closed,PDC,M
...,...,...,...,...,...
14740,Monthly,Arrear,Closed,PDC,M
14741,Monthly,Arrear,Closed,PDC,M
14742,Monthly,Arrear,Closed,PDC,M
14743,Monthly,Arrear,Closed,PDC_E,M


In [43]:
test2.isna().count()

ID                 14745
Frequency          14745
InstlmentMode      14745
LoanStatus         14745
PaymentMode        14745
BranchID           14745
Tenure             14745
AssetCost          14745
AmountFinance      14745
DisbursalAmount    14745
EMI                14745
DisbursalDate      14745
MaturityDAte       14745
AuthDate           14745
AssetID            14745
ManufacturerID     14745
SupplierID         14745
LTV                14745
SEX                14745
AGE                14745
MonthlyIncome      14745
ZiPCODE            14745
dtype: int64

In [44]:
test2.head()

,ID,Frequency,InstlmentMode,LoanStatus,PaymentMode,BranchID,Tenure,AssetCost,AmountFinance,DisbursalAmount,EMI,DisbursalDate,MaturityDAte,AuthDate,AssetID,ManufacturerID,SupplierID,LTV,SEX,AGE,MonthlyIncome,ZiPCODE
0,4,Monthly,Advance,Closed,PDC_E,2,46,480000,365000.00,365000.00,1000.00,2011-12-29,2015-10-05,2011-12-29,3524747,1046.00,22354,75.83,M,50.00,32069.00,473001.00
1,5,Monthly,Advance,Closed,PDC,2,45,480000,285000.00,285000.00,9300.00,2012-04-28,2016-01-01,2012-04-28,4985862,1046.00,22354,57.44,M,35.00,25000.00,473001.00
2,6,Quatrly,Arrear,Closed,PDC,2,48,580000,400000.00,400000.00,35800.00,2013-10-22,2017-09-10,2013-10-22,12881783,1060.00,65929,68.97,M,37.00,23333.33,473001.00
3,25,Half Yearly,Arrear,Closed,Billed,154,36,725000,500000.00,500000.00,52000.00,2013-05-21,2016-02-10,2013-05-21,10613776,1049.00,68597,61.79,M,55.00,91666.67,493558.00
4,119,Quatrly,Arrear,Closed,PDC,194,48,617000,400000.00,400000.00,35400.00,2012-11-30,2016-12-05,2012-11-30,7911110,1049.00,38493,59.09,M,48.00,12500.00,754137.00


In [45]:
le1 = LabelEncoder()
colnames1=['Frequency','InstlmentMode','LoanStatus','PaymentMode','SEX']
colnames1

['Frequency', 'InstlmentMode', 'LoanStatus', 'PaymentMode', 'SEX']

In [46]:
test3=test2
test3[['Frequency1','InstlmentMode1','LoanStatus1','PaymentMode1','SEX1']]=test3[colnames1].apply(le1.fit_transform)

In [47]:
test3.head()

,ID,Frequency,InstlmentMode,LoanStatus,PaymentMode,BranchID,Tenure,AssetCost,AmountFinance,DisbursalAmount,EMI,DisbursalDate,MaturityDAte,AuthDate,AssetID,ManufacturerID,SupplierID,LTV,SEX,AGE,MonthlyIncome,ZiPCODE,Frequency1,InstlmentMode1,LoanStatus1,PaymentMode1,SEX1
0,4,Monthly,Advance,Closed,PDC_E,2,46,480000,365000.00,365000.00,1000.00,2011-12-29,2015-10-05,2011-12-29,3524747,1046.00,22354,75.83,M,50.00,32069.00,473001.00,1,0,1,8,1
1,5,Monthly,Advance,Closed,PDC,2,45,480000,285000.00,285000.00,9300.00,2012-04-28,2016-01-01,2012-04-28,4985862,1046.00,22354,57.44,M,35.00,25000.00,473001.00,1,0,1,6,1
2,6,Quatrly,Arrear,Closed,PDC,2,48,580000,400000.00,400000.00,35800.00,2013-10-22,2017-09-10,2013-10-22,12881783,1060.00,65929,68.97,M,37.00,23333.33,473001.00,2,1,1,6,1
3,25,Half Yearly,Arrear,Closed,Billed,154,36,725000,500000.00,500000.00,52000.00,2013-05-21,2016-02-10,2013-05-21,10613776,1049.00,68597,61.79,M,55.00,91666.67,493558.00,0,1,1,1,1
4,119,Quatrly,Arrear,Closed,PDC,194,48,617000,400000.00,400000.00,35400.00,2012-11-30,2016-12-05,2012-11-30,7911110,1049.00,38493,59.09,M,48.00,12500.00,754137.00,2,1,1,6,1


In [48]:
test4=test3.drop(colnames1, axis=1)
test4.head()

,ID,BranchID,Tenure,AssetCost,AmountFinance,DisbursalAmount,EMI,DisbursalDate,MaturityDAte,AuthDate,AssetID,ManufacturerID,SupplierID,LTV,AGE,MonthlyIncome,ZiPCODE,Frequency1,InstlmentMode1,LoanStatus1,PaymentMode1,SEX1
0,4,2,46,480000,365000.00,365000.00,1000.00,2011-12-29,2015-10-05,2011-12-29,3524747,1046.00,22354,75.83,50.00,32069.00,473001.00,1,0,1,8,1
1,5,2,45,480000,285000.00,285000.00,9300.00,2012-04-28,2016-01-01,2012-04-28,4985862,1046.00,22354,57.44,35.00,25000.00,473001.00,1,0,1,6,1
2,6,2,48,580000,400000.00,400000.00,35800.00,2013-10-22,2017-09-10,2013-10-22,12881783,1060.00,65929,68.97,37.00,23333.33,473001.00,2,1,1,6,1
3,25,154,36,725000,500000.00,500000.00,52000.00,2013-05-21,2016-02-10,2013-05-21,10613776,1049.00,68597,61.79,55.00,91666.67,493558.00,0,1,1,1,1
4,119,194,48,617000,400000.00,400000.00,35400.00,2012-11-30,2016-12-05,2012-11-30,7911110,1049.00,38493,59.09,48.00,12500.00,754137.00,2,1,1,6,1


In [49]:
test4['DisbursalDate1']=test4['DisbursalDate'].map(dt.datetime.toordinal)
test4['MaturityDAte1']=test4['MaturityDAte'].map(dt.datetime.toordinal)
test4['AuthDate1']=test4['AuthDate'].map(dt.datetime.toordinal)
test4.head()

,ID,BranchID,Tenure,AssetCost,AmountFinance,DisbursalAmount,EMI,DisbursalDate,MaturityDAte,AuthDate,AssetID,ManufacturerID,SupplierID,LTV,AGE,MonthlyIncome,ZiPCODE,Frequency1,InstlmentMode1,LoanStatus1,PaymentMode1,SEX1,DisbursalDate1,MaturityDAte1,AuthDate1
0,4,2,46,480000,365000.00,365000.00,1000.00,2011-12-29,2015-10-05,2011-12-29,3524747,1046.00,22354,75.83,50.00,32069.00,473001.00,1,0,1,8,1,734500,735876,734500
1,5,2,45,480000,285000.00,285000.00,9300.00,2012-04-28,2016-01-01,2012-04-28,4985862,1046.00,22354,57.44,35.00,25000.00,473001.00,1,0,1,6,1,734621,735964,734621
2,6,2,48,580000,400000.00,400000.00,35800.00,2013-10-22,2017-09-10,2013-10-22,12881783,1060.00,65929,68.97,37.00,23333.33,473001.00,2,1,1,6,1,735163,736582,735163
3,25,154,36,725000,500000.00,500000.00,52000.00,2013-05-21,2016-02-10,2013-05-21,10613776,1049.00,68597,61.79,55.00,91666.67,493558.00,0,1,1,1,1,735009,736004,735009
4,119,194,48,617000,400000.00,400000.00,35400.00,2012-11-30,2016-12-05,2012-11-30,7911110,1049.00,38493,59.09,48.00,12500.00,754137.00,2,1,1,6,1,734837,736303,734837


In [50]:
test4.shape

(14745, 25)

In [51]:
test5 = test4.drop(['MaturityDAte','AuthDate','DisbursalDate'],axis=1)
test5.head()

,ID,BranchID,Tenure,AssetCost,AmountFinance,DisbursalAmount,EMI,AssetID,ManufacturerID,SupplierID,LTV,AGE,MonthlyIncome,ZiPCODE,Frequency1,InstlmentMode1,LoanStatus1,PaymentMode1,SEX1,DisbursalDate1,MaturityDAte1,AuthDate1
0,4,2,46,480000,365000.00,365000.00,1000.00,3524747,1046.00,22354,75.83,50.00,32069.00,473001.00,1,0,1,8,1,734500,735876,734500
1,5,2,45,480000,285000.00,285000.00,9300.00,4985862,1046.00,22354,57.44,35.00,25000.00,473001.00,1,0,1,6,1,734621,735964,734621
2,6,2,48,580000,400000.00,400000.00,35800.00,12881783,1060.00,65929,68.97,37.00,23333.33,473001.00,2,1,1,6,1,735163,736582,735163
3,25,154,36,725000,500000.00,500000.00,52000.00,10613776,1049.00,68597,61.79,55.00,91666.67,493558.00,0,1,1,1,1,735009,736004,735009
4,119,194,48,617000,400000.00,400000.00,35400.00,7911110,1049.00,38493,59.09,48.00,12500.00,754137.00,2,1,1,6,1,734837,736303,734837


In [52]:
test5.shape

(14745, 22)

In [53]:
dp = test5
dp

,ID,BranchID,Tenure,AssetCost,AmountFinance,DisbursalAmount,EMI,AssetID,ManufacturerID,SupplierID,LTV,AGE,MonthlyIncome,ZiPCODE,Frequency1,InstlmentMode1,LoanStatus1,PaymentMode1,SEX1,DisbursalDate1,MaturityDAte1,AuthDate1
0,4,2,46,480000,365000.00,365000.00,1000.00,3524747,1046.00,22354,75.83,50.00,32069.00,473001.00,1,0,1,8,1,734500,735876,734500
1,5,2,45,480000,285000.00,285000.00,9300.00,4985862,1046.00,22354,57.44,35.00,25000.00,473001.00,1,0,1,6,1,734621,735964,734621
2,6,2,48,580000,400000.00,400000.00,35800.00,12881783,1060.00,65929,68.97,37.00,23333.33,473001.00,2,1,1,6,1,735163,736582,735163
3,25,154,36,725000,500000.00,500000.00,52000.00,10613776,1049.00,68597,61.79,55.00,91666.67,493558.00,0,1,1,1,1,735009,736004,735009
4,119,194,48,617000,400000.00,400000.00,35400.00,7911110,1049.00,38493,59.09,48.00,12500.00,754137.00,2,1,1,6,1,734837,736303,734837
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14740,143396,143,35,530016,419616.00,419616.00,14723.00,4276747,1046.00,22094,79.17,40.23,45084.91,122001.00,1,1,1,6,1,734562,735639,734562
14741,143397,32,36,595000,446500.00,446500.00,15491.00,4746586,2721.00,61196,75.04,40.23,45084.91,560020.00,1,1,1,6,1,734603,735693,734603
14742,143398,32,36,595000,446500.00,446500.00,15491.00,4746588,2721.00,61196,75.04,40.23,45084.91,560020.00,1,1,1,6,1,734603,735693,734603
14743,143399,246,12,400000,280000.00,280000.00,25350.00,11719081,1046.00,63565,70.00,40.23,45084.91,335001.00,1,1,1,8,1,735074,735455,735075


In [54]:
dp=scaling.fit_transform(dp)

In [55]:
dp_pred = classifier.predict(dp)

In [56]:
unique_elements, counts_elements = np.unique(dp_pred, return_counts=True)
print(np.asarray((unique_elements, counts_elements)))

[[' > 48 Months' '12-18 Months' '18-24 Months' '24-30 Months'
  '30-36 Months' '36-48 Months' 'No Top-up Service']
 [35 8 15 21 3 100 14563]]


In [57]:
test6 = test5
test6['df_pred'] = dp_pred
test6.head()

,ID,BranchID,Tenure,AssetCost,AmountFinance,DisbursalAmount,EMI,AssetID,ManufacturerID,SupplierID,LTV,AGE,MonthlyIncome,ZiPCODE,Frequency1,InstlmentMode1,LoanStatus1,PaymentMode1,SEX1,DisbursalDate1,MaturityDAte1,AuthDate1,df_pred
0,4,2,46,480000,365000.00,365000.00,1000.00,3524747,1046.00,22354,75.83,50.00,32069.00,473001.00,1,0,1,8,1,734500,735876,734500,No Top-up Service
1,5,2,45,480000,285000.00,285000.00,9300.00,4985862,1046.00,22354,57.44,35.00,25000.00,473001.00,1,0,1,6,1,734621,735964,734621,No Top-up Service
2,6,2,48,580000,400000.00,400000.00,35800.00,12881783,1060.00,65929,68.97,37.00,23333.33,473001.00,2,1,1,6,1,735163,736582,735163,No Top-up Service
3,25,154,36,725000,500000.00,500000.00,52000.00,10613776,1049.00,68597,61.79,55.00,91666.67,493558.00,0,1,1,1,1,735009,736004,735009,No Top-up Service
4,119,194,48,617000,400000.00,400000.00,35400.00,7911110,1049.00,38493,59.09,48.00,12500.00,754137.00,2,1,1,6,1,734837,736303,734837,No Top-up Service


In [58]:
test7 = test6[['ID','df_pred']]

In [59]:
test7

,ID,df_pred
0,4,No Top-up Service
1,5,No Top-up Service
2,6,No Top-up Service
3,25,No Top-up Service
4,119,No Top-up Service
...,...,...
14740,143396,No Top-up Service
14741,143397,> 48 Months
14742,143398,> 48 Months
14743,143399,No Top-up Service


In [60]:
test7 = test7.rename(columns={'df_pred':'Top-up Month'})

In [61]:
test7

,ID,Top-up Month
0,4,No Top-up Service
1,5,No Top-up Service
2,6,No Top-up Service
3,25,No Top-up Service
4,119,No Top-up Service
...,...,...
14740,143396,No Top-up Service
14741,143397,> 48 Months
14742,143398,> 48 Months
14743,143399,No Top-up Service


In [62]:
test7.to_csv('LTFS_prediction_standardScaler.csv',index=False)

### Crossvalidation score for XGB classifier

In [235]:
score=cross_val_score(classifier,X_train,y_train,cv=10)

c:\users\vagundu\appdata\local\r-mini~1\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:00:19] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:01:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:02:39] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:03:44] WARNING: C:/Users/Administrator/workspace/xgboost-win64_

In [236]:
score

array([0.82798834, 0.82725948, 0.82567948, 0.82661668, 0.82776216,
       0.82713735, 0.82651255, 0.82672082, 0.82776216, 0.82713735])

### Using RandomizedSearchCV model

In [227]:
params={
 "learning_rate"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
 "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15],
 "min_child_weight" : [ 1, 3, 5, 7 ],
 "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
 "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ]
}

In [230]:
random_search=RandomizedSearchCV(classifier,param_distributions=params,n_iter=5,scoring='roc_auc',n_jobs=-1,cv=5,verbose=3)

In [231]:
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

In [ ]:
start_time = timer(None)
random_search.fit(X_train, y_train)
timer(start_time) 

In [ ]:
random_search.best_estimator_
random_search.best_params_

### Decision Tree classifier for Multi class classification

In [241]:
dtree_model = DecisionTreeClassifier(max_depth = 2).fit(X_train, y_train) 
dtree_predictions = dtree_model.predict(X_test) 

In [243]:
cm = confusion_matrix(y_test, dtree_predictions) 
cm

array([[    0,     0,     0,     0,     0,     0,  2012],
       [    0,     0,     0,     0,     0,     0,   270],
       [    0,     0,     0,     0,     0,     0,   630],
       [    0,     0,     0,     0,     0,     0,   886],
       [    0,     0,     0,     0,     0,     0,   736],
       [    0,     0,     0,     0,     0,     0,   873],
       [    0,     0,     0,     0,     0,     0, 26604]], dtype=int64)

### Naive Bayes Classification

In [87]:
from sklearn.naive_bayes import GaussianNB 
gnb = GaussianNB().fit(X_train, y_train) 
gnb_predictions = gnb.predict(X_test) 
  
# accuracy on X_test 
accuracy = gnb.score(X_test, y_test) 
print(accuracy)
  
# creating a confusion matrix 
cm = confusion_matrix(y_test, gnb_predictions)
cm

0.1129611695979507


array([[ 1527,    16,     0,     7,    38,   414,    86],
       [   41,    40,    83,    61,    12,    17,     4],
       [   50,    40,   121,   320,    19,    34,    10],
       [   62,    28,    86,   536,   121,    36,     6],
       [   61,    20,    35,   243,   304,    67,    12],
       [  186,    19,    11,   129,   316,   226,    25],
       [12978,  1059,  1774,  4087,  2714,  3068,   862]], dtype=int64)

In [88]:
gnb_pred = gnb.predict(dp)

In [90]:
gnb_pred.shape

(14745,)

In [91]:
gnb_pred

array(['No Top-up Service', 'No Top-up Service', 'No Top-up Service', ...,
       ' > 48 Months', ' > 48 Months', ' > 48 Months'], dtype='<U17')

In [98]:
gnb_df = pd.DataFrame(gnb_pred, test7.ID, columns =['Top-up Month'] )

In [99]:
gnb_df

,Top-up Month
ID,
4,No Top-up Service
5,No Top-up Service
6,No Top-up Service
25,No Top-up Service
119,No Top-up Service
...,...
143396,> 48 Months
143397,> 48 Months
143398,> 48 Months


In [101]:
gnb_df.to_csv('LNTFS_prediction_GNB.csv')

### SVM

In [102]:
from sklearn.svm import SVC 
svm_model_linear = SVC(kernel = 'linear', C = 1).fit(X_train, y_train) 
svm_predictions = svm_model_linear.predict(X_test) 
  
# model accuracy for X_test   
accuracy = svm_model_linear.score(X_test, y_test) 
  
# creating a confusion matrix 
cm = confusion_matrix(y_test, svm_predictions)
cm

array([[    0,     0,     0,     0,     0,     0,  2088],
       [    0,     0,     0,     0,     0,     0,   258],
       [    0,     0,     0,     0,     0,     0,   594],
       [    0,     0,     0,     0,     0,     0,   875],
       [    0,     0,     0,     0,     0,     0,   742],
       [    0,     0,     0,     0,     0,     0,   912],
       [    0,     0,     0,     0,     0,     0, 26542]], dtype=int64)

In [110]:
accuracy

0.8291524788354003

In [118]:
svm_predic= svm_model_linear.predict(dp)

In [119]:
svm_df = pd.DataFrame(svm_predic, test7.ID, columns =['Top-up Month'] )
gm_df.to_csv('LTFS_Predictions_svm.csv')

### Gaussian Mixure predictions on Test data and saving to .csv

In [103]:
gm_pred= gm.predict(dp)

In [112]:
gm_df = pd.DataFrame(gm_pred, test7.ID, columns =['Top-up Month'] )
gm_df.to_csv('LTFS_Predictions_gaussianMixture.csv')

### Neural Network MLPClassifier

In [113]:
from sklearn.neural_network import MLPClassifier
snn_classifier = MLPClassifier()
snn_classifier.fit(X_train, y_train)
snn_predictions = snn_classifier.predict(dp)

In [114]:
snn_df = pd.DataFrame(snn_predictions, test7.ID, columns =['Top-up Month'] )
snn_df.to_csv('LTFS_Predictions_snn.csv')

### KNN

In [115]:
from sklearn.neighbors import KNeighborsClassifier
knn_classifier = KNeighborsClassifier()
knn_classifier.fit(X_train, y_train)
knn_predictions = knn_classifier.predict(dp)

In [116]:
knn_df = pd.DataFrame(knn_predictions, test7.ID, columns =['Top-up Month'] )
knn_df.to_csv('LTFS_Predictions_knn.csv')